<a href="https://colab.research.google.com/github/SohaHussain/Machine-Learning/blob/main/Lexical%20Processing/Norvig_Spell_Corrector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Spell Corrector

In [1]:
import re
from collections import Counter

In [2]:
# to extract all words

# \w+ -> regular expression for words
# converting words to lowercase and then tokenizing them
def words(doc):
  return re.findall(r'\w+',doc.lower())

A seed document acts like a lookup dictionary for a spell corrector since it contains the correct spellings of each word.

In [3]:
# creating a frequency table of all words present in the seed document

all_words = Counter(words(open('Seed document.txt').read()))
print(len(words(open('Seed document.txt').read())))
print(len(all_words))

1057538
30643


In [6]:
# checking frequency of a random word

all_words['are']

3475

In [7]:
# 5 most frequent words

all_words.most_common(5)

[('the', 77235), ('of', 38359), ('and', 36781), ('to', 27547), ('in', 21349)]

In [8]:
# creates edits that are one edit away from 'word'

def edits_one(word):
  alpha = 'abcdefghijklmnopqrstuvwxyz'
  splits = [(word[:i], word[i:]) for i in range (len(word)+1) ]
  deletes = [left + right[1:]    for left,right in splits if right]
  inserts = [left + c + right    for left,right in splits for c in alpha]
  replaces = [left+c+right[1:]   for left,right in splits if right for c in alpha]
  transposes = [left+right[1]+right[0]+right[2:] for left,right in splits if len(right)>1]
  return set(deletes+inserts+replaces+transposes)

In [9]:
# creates edits that are two edits away from 'word'

def edits_two(word):
  return (e2 for e1 in edits_one(word) for e2 in edits_one(e1))

In [24]:
# The known() function filters out the valid English word from a list of given words. 
# It uses the frequency distribution as a dictionary that was created using the seed document. 
# If the words created using edits_one() and edits_two() are not in the dictionary, they’re discarded.

def known(words):
  return set( word for word in words if word in all_words)

the function possible_corrections() returns a list of all the potential words that can be the correct alternative spelling.

It first checks if the word is correct or not, i.e. if the word typed by the user is a present in the dictionary or not. If the word is present, it returns no spelling suggestions since it is already a correct dictionary word.

If the user types a word which is not a dictionary word, then it creates a list of all the known words that are one edit distance away. If there are no valid words in the list created by edits_one() only then this function fetches a list of all known words that are two edits away from the input word

If there are no known words that are two edits away, then the function returns the  original input word. This means that there are no alternatives that the spell corrector could find. Hence, it simply returns the original word.

In [18]:
def pos_corrections(word):
  return (known([word]) or known(edits_one(word)) or known(edits_two(word)) or [word])

In [12]:
def prob(word, n=sum(all_words.values())):
  return all_words[word]/n

In [13]:
print(len(set(edits_one("monney"))))
print(edits_one("monney"))

336
{'monnel', 'ponney', 'money', 'monyney', 'mfonney', 'mocnney', 'monner', 'monneyw', 'qonney', 'mononey', 'monnny', 'mvnney', 'mzonney', 'monneye', 'monnjey', 'monfey', 'mosnney', 'montney', 'monnely', 'imonney', 'monneyh', 'monncy', 'monnehy', 'moxnney', 'monnpy', 'monnqy', 'monnyy', 'molnney', 'monyey', 'monneyj', 'monoey', 'monaey', 'monnegy', 'monneuy', 'dmonney', 'moynney', 'monnzey', 'monkney', 'monneqy', 'monrney', 'monnxey', 'monneyc', 'morney', 'mownney', 'mfnney', 'monndey', 'monneyi', 'monnpey', 'monnye', 'mkonney', 'monngey', 'mtonney', 'mozney', 'monfney', 'monnsey', 'monneey', 'monneyq', 'emonney', 'jmonney', 'lmonney', 'monnecy', 'monnef', 'mnney', 'fonney', 'monuney', 'mornney', 'monpney', 'sonney', 'moeney', 'monnexy', 'cmonney', 'honney', 'pmonney', 'mconney', 'qmonney', 'monnen', 'mofney', 'mgonney', 'maonney', 'monnkey', 'moyney', 'meonney', 'mnonney', 'moneey', 'molney', 'mlonney', 'monnezy', 'monneya', 'zmonney', 'msonney', 'mofnney', 'moncey', 'monnwey', 'uonn

In [14]:
print(known(edits_one("monney")))

{'monkey', 'money'}


In [15]:
print(len(set(edits_two("monney"))))
print(known(edits_one("monney")))

51013
{'monkey', 'money'}


In [19]:
print(pos_corrections("monney"))

{'monkey', 'money'}


In [20]:
print(prob("money"))
print(prob("monkey"))

0.00028746011963636295
3.782369995215302e-06


In [22]:
def spell_check(word):
  "Print the most probable spelling correction for `word` out of all the `possible_corrections`"
  correct_word = max(pos_corrections(word), key=prob)
  if correct_word != word:
    return "Did you mean " + correct_word + "?"
  else:
    return "Correct spelling."

In [25]:
# test spell check
print(spell_check("leaarn"))

Did you mean learn?
